In [1]:
import torch
import numpy as np
import random
import gc

from src.model import get_tokenizer
from src.preprocessing import prepare_dataset
from src.dataset import get_paraphrased_dataset, get_model_generated_dataset, get_original_dataset
from src.utility import pprint_messages

In [2]:
# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
torch.cuda.empty_cache()
gc.collect()

1121

In [4]:
model_name = "amd/AMD-OLMo-1B-SFT"

In [5]:
tokenizer = get_tokenizer(model_name)

In [6]:
original_dataset = get_original_dataset(sample_size=None)
pprint_messages(original_dataset[0]["messages"])

==== User Message ====
Can brain cells move? By movement I mean long distance migration (preferably within the brain only).

==== Assistant Message ====
The question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.
However, after sifting through various publications, the answer to the question is actually remarkably simple: Yes, brain cells migrate.
In  the adult brain glial cells migrate in the brain (Klämbt, 2009). Glial cells are involved in a myriad of functions, but a notable example of migrating glial cells are the oligodendrocytes that migrate relative long distances to find their target axons onto which they wrap themselves to form the insulating myelin sheath (Tsai and Miller, 2002).


In [7]:
paraphrased_dataset = get_paraphrased_dataset(sample_size=None)
pprint_messages(paraphrased_dataset[0]["paraphrased_messages"])

==== User Message ====
Are brain cells capable of moving? Specifically, I'm referring to long-distance migration, ideally occurring within the brain.

==== Assistant Message ====
The inquiry is quite extensive, and it’s important to recognize that the brain is made up not only of neurons but also of glial cells (supporting cells) and pre-mitotic neuronal stem cells. Additionally, as critical colleagues in research have pointed out, the developmental stage is crucial, since the developing embryonic brain is significantly different from the fully developed adult brain. Nonetheless, after reviewing various studies, the answer to the question turns out to be surprisingly straightforward: Yes, brain cells do migrate. In the adult brain, glial cells are known to migrate (Klämbt, 2009). These glial cells perform numerous functions, with a prominent example being oligodendrocytes, which migrate over considerable distances to locate their target axons and encase them to create the insulating my

In [8]:
model_generated_dataset = get_model_generated_dataset(model_name=model_name, sample_size=None)
pprint_messages(model_generated_dataset["model_generated_messages"][0])

==== User Message ====
Are brain cells capable of moving? Specifically, I'm referring to long-distance migration, ideally occurring within the brain.

==== Assistant Message ====
Brain cells, such as neurons, are indeed capable of moving. This movement is crucial for various functions, including communication between neurons and the formation of neural networks. However, the extent of brain cell movement is limited by the physical constraints of the brain's structure.

Neurons are highly organized and interconnected, forming complex networks that enable the brain to process and interpret information. The movement of neurons within these networks is facilitated by the cytoskeleton, which provides structural support and allows for the movement of the cell body.

While brain cells can move within the brain, the movement is typically limited to short distances and is not as extensive as the movement of neurons in other parts of the body. The brain's structure and the limitations of its phy

In [9]:
model_generated_dataset["model_generated_messages"][0]

[{'content': "Are brain cells capable of moving? Specifically, I'm referring to long-distance migration, ideally occurring within the brain.",
  'role': 'user'},
 {'content': "Brain cells, such as neurons, are indeed capable of moving. This movement is crucial for various functions, including communication between neurons and the formation of neural networks. However, the extent of brain cell movement is limited by the physical constraints of the brain's structure.\n\nNeurons are highly organized and interconnected, forming complex networks that enable the brain to process and interpret information. The movement of neurons within these networks is facilitated by the cytoskeleton, which provides structural support and allows for the movement of the cell body.\n\nWhile brain cells can move within the brain, the movement is typically limited to short distances and is not as extensive as the movement of neurons in other parts of the body. The brain's structure and the limitations of its ph

In [ ]:
model_generated_dataset["messages"][0][1]["content"]

In [ ]:
original_dataset_config = get_dataset_config(model, sft_messages_key="messages")
original_dataset_config

In [ ]:
paraphrased_dataset_config = get_dataset_config(model, sft_messages_key="paraphrased_messages")
paraphrased_dataset_config

In [ ]:
sample_size = 5

In [ ]:
original_dataset_tokenized = prepare_dataset(dataset=orig, dataset_config=original_dataset_config, tokenizer=tokenizer, sample_size=sample_size)

In [ ]:
paraphrased_dataset_tokenized = prepare_dataset(dataset=dataset, dataset_config=paraphrased_dataset_config, tokenizer=tokenizer, sample_size=sample_size)

In [ ]:
model_generated_dataset_tokenized = prepare_dataset(dataset=model_generated_dataset, dataset_config=original_dataset_config, tokenizer=tokenizer, sample_size=sample_size)

In [ ]:
gradients_sample_0 = get_gradients(model, original_dataset_tokenized[0], device=device)

In [ ]:
for key, value in gradients_sample_0.items():
    print(f"{key}: {value.shape}")

In [ ]:
original_dataset_tokenized[0]["input_ids"]

In [ ]:
decoded_input = tokenizer.decode(token_ids=original_dataset_tokenized[0]["input_ids"])
decoded_input

In [ ]:
decoded_output = tokenizer.decode(token_ids=[(tokenizer.pad_token_id if token == -100 else token) for token in original_dataset_tokenized[0]["labels"]])
decoded_output

In [ ]:
original_dataset_tokenized[0]["input_ids"]

In [ ]:
output = model.generate(original_dataset_tokenized[0]["input_ids"].reshape(1, -1), max_new_tokens=100)

In [ ]:
dataset[0]["messages"][0]["content"]

In [ ]:
inputs = tokenizer(dataset[0]["messages"][0]["content"], return_tensors="pt")

In [ ]:
generate_ids = model.generate(inputs.input_ids, max_new_tokens=100, return_full_text=False)

In [ ]:
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [ ]:
print(output)

In [ ]:
tokenizer.batch_decode(output, skip_special_tokens=True, clean_up_tokenzation_spaces=False)

In [ ]:
dataset[0]